In [1]:
import json
from model import ChatGLMModel
from transformers import AutoTokenizer
from tqdm import *
import torch
from transformers.generation.logits_process import LogitsProcessor
from transformers.generation.utils import LogitsProcessorList, StoppingCriteriaList, GenerationConfig, ModelOutput
import pandas as pd

model = ChatGLMModel()
lora_model = model.lora_model.from_pretrained(model.model, 'lora_r16_p2_0524_v0/epoch_3_epoch_acc_0.1217').eval()
max_length = 1000
model_path = '../../model/chatglm-6b'
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
with open('../data/instruction_testA_with_node.json', 'r') as f:
    test_data = json.load(f)
# page_node_df = pd.read_csv('../data/0523/page_node.csv')
# test_df = pd.read_csv('../data/0523/test_df.csv')
prefix = "要在该网页上完成这个指令,用json格式生成操作和相应参数:"


class InvalidScoreLogitsProcessor(LogitsProcessor):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        if torch.isnan(scores).any() or torch.isinf(scores).any():
            scores.zero_()
            scores[..., 5] = 5e4
        return scores


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


Global seed set to 0
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


trainable params: 7340032 || all params: 6180626432 || trainable%: 0.11875870643139365


In [5]:
batch_size = 3

res = []
error_count = 0
total_count = 0
for page_idx, data in enumerate(test_data):
    page, page_node = data['page_source'], data['node_name']
    page_res = {'page_source': page, 'instruction_detail':[]}
    for idx in trange(0, len(data['instruction']), batch_size):
    # for idx in trange(0, batch_size, batch_size):
        if len(data['instruction'][idx:idx+batch_size]) <= 0:
            break
        row_texts, batch_instruction = [], []
        for instruction in data['instruction'][idx:idx+batch_size]:
            row_text = '网页:{}。该网页可操作的元素有:{}。指令:{}。{}'.format(page, page_node, instruction, prefix)
            row_texts.append(row_text)
            batch_instruction.append(instruction)
        input_ids = tokenizer(row_texts, return_tensors='pt', padding='longest')['input_ids'].to(lora_model.device)
        logits_processor = LogitsProcessorList()
        logits_processor.append(InvalidScoreLogitsProcessor())
        gen_kwargs = {"max_new_tokens": max_length, "num_beams": 3, "do_sample": False, "top_p": 0.8,
                                "temperature":0.9, 'repetition_penalty':1.1, "logits_processor": logits_processor,} 
        output = lora_model.generate(input_ids=input_ids, **gen_kwargs)
        for i in range(input_ids.shape[0]):
            row_res = tokenizer.decode(output[i])
            try:
                row_res = eval(row_res[row_res.index(prefix)+len(prefix):].strip(' '))
                if not isinstance(row_res, dict):
                    error_count += 1
                    print(111, row_res)
                    row_res = {}
                elif any(isinstance(v, list)==False for v in row_res.values()):
                    error_count += 1
                    print(222, row_res)
                    row_res = {}
            except Exception as e:
                print('error: ', e)
                print(row_res[row_res.index(prefix)+len(prefix):].strip(' '))
                error_count += 1
                row_res = {}
            page_res['instruction_detail'].append({'instruction':batch_instruction[i],
                                                'key-value': row_res})
            # print(row_res)
            total_count += 1
    res.append(page_res)
    print('page_idx:{}  total_count:{}  error_count:{}'.format(page_idx, total_count, error_count))

100%|██████████| 10/10 [01:55<00:00, 11.58s/it]


page_idx:0  total_count:30  error_count:0


 10%|█         | 1/10 [00:16<02:27, 16.34s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   20 │   │   logits_processor.append(InvalidScoreLogitsProcessor())                              │
│   21 │   │   gen_kwargs = {"max_new_tokens": max_length, "num_beams": 4, "do_sample": False,     │
│   22 │   │   │   │   │   │   │   │   "temperature":0.9, 'repetition_penalty':1.1, "logits_pro    │
│ ❱ 23 │   │   output = lora_model.generate(input_ids=input_ids, **gen_kwargs)                     │
│   24 │   │   for i in range(input_ids.shape[0]):                                                 │
│   25 │   │   │   row_res = tokenizer.decode(output[i])                                           │
│   26 │   │   │   try:                                                                            │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/peft/peft_model.py:731 in generate                  │
│                                                                                                  │
│    728 │   │   self.base_model.prepare_inputs_for_generation = self.prepare_inputs_for_generati  │
│    729 │   │   try:                                                                              │
│    730 │   │   │   if not isinstance(peft_config, PromptLearningConfig):                         │
│ ❱  731 │   │   │   │   outputs = self.base_model.generate(**kwargs)                              │
│    732 │   │   │   else:                                                                         │
│    733 │   │   │   │   if "input_ids" not in kwargs:                                             │
│    734 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati  │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/torch/autograd/grad_mode.py:27 in decorate_context  │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/transformers/generation/utils.py:1490 in generate   │
│                                                                                                  │
│   1487 │   │   │   │   **model_kwargs,                                                           │
│   1488 │   │   │   )                                                                             │
│   1489 │   │   │   # 13. run beam search                                                         │
│ ❱ 1490 │   │   │   return self.beam_search(                                                      │
│   1491 │   │   │   │   input_ids,                                                                │
│   1492 │   │   │   │   beam_scorer,                                                              │
│   1493 │   │   │   │   logits_processor=logits_processor,  

In [6]:
res

[{'page_source': 'qq音乐',
  'instruction_detail': [{'instruction': '查一下女性的歌手列表',
    'key-value': {'搜索': ['按钮', '', '点击'], '女': ['单选框', '', '点击']}},
   {'instruction': '查找一下说唱风格的歌手列表',
    'key-value': {'搜索': ['按钮', '', '点击'], '说唱': ['单选框', '', '点击']}},
   {'instruction': '查询韩国地区的歌手列表',
    'key-value': {'搜索': ['按钮', '', '点击'], '韩国': ['单选框', '', '点击']}},
   {'instruction': '查找轻音乐风格的歌手列表',
    'key-value': {'搜索': ['按钮', '', '点击'], '轻音乐': ['单选框', '', '点击']}},
   {'instruction': '请查询R&B风格的歌手列表',
    'key-value': {'搜索': ['按钮', '', '点击'], 'R&B': ['单选框', '', '点击']}},
   {'instruction': '请查找组合形式的歌手列表',
    'key-value': {'搜索': ['按钮', '', '点击'], '组合': ['单选框', '', '点击']}},
   {'instruction': '查询日本地区的男性的爵士风格的歌手列表',
    'key-value': {'搜索': ['按钮', '', '点击'],
     '日本': ['单选框', '', '点击'],
     '男': ['单选框', '', '点击'],
     '爵士': ['单选框', '', '点击']}},
   {'instruction': '查找出港台地区的组合形式的爵士风格的歌手列表',
    'key-value': {'搜索': ['按钮', '', '点击'],
     '港台': ['单选框', '', '点击'],
     '组合': ['单选框', '', '点击'],
     '爵

In [ ]:
#### 别删

import json
import pandas as pd
with open('sub/chatglm6b_r16_epoch_40_with_node_prompt_test_mid_sub_0526_v0.json', 'r') as f:
    sub = json.load(f)
page_node_df = pd.read_csv('../data/0524/page_node_p2_df.csv')

def transform_result(sub):
    res = []
    for page_res in sub:
        new_page_res = {'page_source': page_res['page_source']}
        instruction_detail = []
        for detail in page_res['instruction_detail']:
            instruction, key_value = detail['instruction'], detail['key-value']
            new_detail = {'instruction': instruction}
            new_key_value = {}
            for k, v in key_value.items():
                try:
                    new_key_value[k] = {'dom_type':v[0], 'value':v[1], 'action':v[2]}
                except:
                    print(instruction)
                    print(key_value)
                    print(k, v)
                    continue
            new_detail['key-value'] = new_key_value
            instruction_detail.append(new_detail)
        new_page_res['instruction_detail'] = instruction_detail
        res.append(new_page_res)
    return res

res = transform_result(sub)

In [9]:
with open('sub/chatglm6b_r16_epoch_40_with_node_prompt_sub_0526_v0.json', 'w') as f:
    json.dump(res, f)

In [12]:
res[19]

{'page_source': '顺丰快递查运费时效',
 'instruction_detail': [{'instruction': '请检索从四川省成都市从化区寄到美国Yuexiu的快递时效，其中寄件时间是2023-01-10上午09:12:14，包裹重量为0.6kg',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '始发地': {'dom_type': '输入框', 'value': '四川省-成都市-从化区', 'action': '输入'},
    '目的地': {'dom_type': '输入框', 'value': '美国-Yuexiu', 'action': '输入'},
    '寄件时间': {'dom_type': '输入框',
     'value': '2023-01-10 09:12:14',
     'action': '输入'},
    '包裹重量': {'dom_type': '输入框', 'value': '0.600', 'action': '输入'}}},
  {'instruction': '查找从香港荔湾区发往加拿大Zengcheng的快递运费，其中包裹重量为0.8kg，寄件时间是2023-01-17下午12:23:45',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '始发地': {'dom_type': '输入框', 'value': '香港-荔湾区', 'action': '输入'},
    '目的地': {'dom_type': '输入框', 'value': '加拿大-Zengcheng', 'action': '输入'},
    '寄件时间': {'dom_type': '输入框',
     'value': '2023-01-17 12:23:45',
     'action': '输入'},
    '包裹重量': {'dom_type': '输入框', 'value': '0.800', 'action': '输入'}}},
  {'instruction': '请检索从福建